About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
    1: Fake news
    0: real News

In [ ]:
import numpy as np
import pandas as pd
import re     #regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer      #convert text into numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords in english

print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data pre-processing

In [ ]:
# loading the dataset to a pandas dataframe

news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='skip', engine='python')


In [ ]:
news_dataset.shape

(2179, 5)

In [ ]:
# print the first 5 rows of the dataframe

news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# counting the number of missing values in the dataset

news_dataset.isnull().sum()

,0
id,0
title,57
author,227
text,6
label,0


In [ ]:
# replacing the null values with empty string

news_dataset = news_dataset.fillna('')

In [ ]:
#merging the author name and news title

news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
2174    REALdeal These People Reversed Their Diabetes ...
2175                             Yves Smith Links 11/1/16
2176    Madeline Flash-Mob Meditations To Occur At Vot...
2177    Ian Greenhalgh Trump’s grandfather was a pimp ...
2178    Warner Todd Huston Postman Arrested for Feedin...
Name: content, Length: 2179, dtype: object


In [ ]:
#separating the data and label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
2174  2174    These People Reversed Their Diabetes In 30 Days   
2175  2175                                      Links 11/1/16   
2176  2176  Flash-Mob Meditations To Occur At Voting Stati...   
2177  2177  Trump’s grandfather was a pimp and tax evader;...   
2178  2178  Postman Arrested for Feeding Dog Nail-Filled M...   

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        Daniel J. Flynn  Ever get the feeling your life circ

Stemming

stemming is the process of reducing a word to its root word

eg: actor,actress,acting---> root word is act

prefix and suffix will be removed and root word is captured for better perormance

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
     stemmed_content = re.sub('[^a-zA-Z]',' ',content)   #removes numbers punctuations etc .replaced by space #re is regular expression library
     stemmed_content = stemmed_content.lower()           #converts to lowercase
     stemmed_content = stemmed_content.split()           #converting and splitted to a list
     stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]   #removes stopwords
     stemmed_content = ' '.join(stemmed_content)         #joins all the words
     return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
2174                     realdeal peopl revers diabet day
2175                                       yve smith link
2176     madelin flash mob medit occur vote station elect
2177    ian greenhalgh trump grandfath pimp tax evad f...
2178    warner todd huston postman arrest feed dog nai...
Name: content, Length: 2179, dtype: object


In [ ]:
#separating the data and label

X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'madelin flash mob medit occur vote station elect'
 'ian greenhalgh trump grandfath pimp tax evad father member kkk'
 'warner todd huston postman arrest feed dog nail fill meatbal']


In [ ]:
print(Y)

[1 0 1 ... 1 1 0]


In [ ]:
#converting text into numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 106)	0.2772552757629272
  (0, 873)	0.35735215877683657
  (0, 1037)	0.25912980083835285
  (0, 1278)	0.33922668385226223
  (0, 1346)	0.270364290013619
  (0, 1741)	0.2501592344789762
  (0, 2450)	0.23203375955440184
  (0, 2686)	0.2462695717182597
  (0, 2989)	0.30134999405828655
  (0, 3083)	0.33922668385226223
  (0, 4591)	0.26439504664283403
  (0, 5369)	0.2953807506875015
  (1, 538)	0.2977232361676979
  (1, 681)	0.15676790320857817
  (1, 788)	0.3897493294255797
  (1, 983)	0.20071554829941501
  (1, 1270)	0.2874166406386227
  (1, 1942)	0.692462967670144
  (1, 2394)	0.19539054738074088
  (1, 5694)	0.30708740634600384
  (2, 1032)	0.3150152072280342
  (2, 1096)	0.4782700139424647
  (2, 1901)	0.3643120003406156
  (2, 2111)	0.3467755921957265
  (2, 3320)	0.49334888706557395
  :	:
  (2176, 3269)	0.369230410390172
  (2176, 3369)	0.4170319792367087
  (2176, 3630)	0.4170319792367087
  (2176, 4948)	0.369230410390172
  (2176, 5554)	0.2569760728502728
  (2177, 1737)	0.36233878667620556
  (2177, 184

Splitting dataset into training and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

Training the model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation



accuracy score

In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of training data : ', training_data_accuracy)

Accuracy score of training data :  0.9707401032702238


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)


In [ ]:
print('Accuracy score of testing data : ', testing_data_accuracy)

Accuracy score of testing data :  0.9357798165137615


Making a predictive system

In [ ]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake
